<a href="https://colab.research.google.com/github/edennova501/EXOTIC/blob/main/EXOTIC_Standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXOTIC Standard

<font face="Helvetica, Arial, Sans-Serif">


Run EXOTIC on images from an Exoplanet Watch Telescope.

* User Guide: <a href="https://exoplanets.jpl.nasa.gov/exoplanet-watch/exotic/user-guide/#3-1" target="_blank">How to get data for use in EXOTIC Standard</a>.
* Have your own images? <a href="https://colab.research.google.com/drive/1ApiY_HfyAjwk5i-3KtFKP1XQMq8Vmqcb">Use EXOTIC Advanced</a>.

Click the Screen Shot 2022-09-08 at 10.19.48 AM.png or Screen Shot 2022-09-08 at 10.19.58 AM.png icons below to run the code for each step.


</font>

---

## Step 1: Load EXOTIC and mount Google Drive

<font face="Helvetica, Arial, Sans-Serif">


🕔&nbsp;&nbsp;<i>Estimated time: 2 minutes</i>
<br />

<br />


This step will:


<ol type="a">
<li class="step">Import required EXOTIC libraries</li>
<li class="step">Connect to your Google Drive to access your .FITS images</li>
</ol>

<table><tr><td bgcolor="#fee9e9">
  <font size=2 color="#c02020">Important: there will be two popups you must agree to.
  <ul><li><font size=2>First popup is titled "Warning: This notebook was not authored by Google.". Click "Run Anyway".</font></li>
  <li><font size=2>Second popup is asking for permission to access your Google Drive files. Click "Connect to Google Drive" and follow the prompts.</font></li></ul></font>
</td></tr></table>
<br />

</font>

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8><b>Load EXOTIC libraries and mount Google Drive</b></font>

##############################################################
%%capture step_capture --no-display
# Comment the above statement out to see debugging information
##############################################################

##############################################################
#
# NOTE TO EXOTIC USER:
#
#   • To hide this code, double-click the title above ("Load telescope images"),
#     or click the arrow to the left of the title.
#
#   • Editing this code will only affect your local instance.
#     Reload to revert your changes.
#
##############################################################

# Import display libraries to allow html, css, and javascript for styling and interaction
from IPython.display import display, HTML, Javascript
display(HTML('<p class="bookend">START: Importing necessary software libraries</p>'))
display(HTML('<p class="hidden">Loading styles, please wait...</p>'))

# Install EXOTIC Colab interface code to provide styling and interaction features
# !pip install git+https://github.com/alienlifeform/exotic-colab.git --upgrade
!pip install --upgrade setuptools~=66.1
!pip install --upgrade --use-deprecated=legacy-resolver git+https://github.com/alienlifeform/exotic-colab.git
!pip install importlib-metadata
from exoticcolab.display import setupDisplay, testImplementation, displayStep, makeContainer, downloadButton, appendToContainer, appendStepToContainer, expandableSection, expandableSectionCustom, hideWarning, showProgress, resize_colab_cell
# Set up custom Colab styles and interactions
setupDisplay()
# Improve how colab handles long code output fields
get_ipython().events.register('pre_run_cell', resize_colab_cell)

# Show progress
display(HTML('<ul class="step_container_1a"></ul>'))
appendStepToContainer('.step_container_1a','Styles loaded, importing libraries...')
showProgress(.3)

# Import graphing libraries
appendStepToContainer('.step_container_1a','(1/5) Bokeh.io')

import bokeh.io
from bokeh.io import output_notebook

if bokeh.__version__ != "2.4.3":
  !pip install bokeh==2.4.3
  hideWarning()
else:
  showProgress(.3)

# Import EXOTIC
appendStepToContainer('.step_container_1a','(2/5) EXOTIC <span class="comment">(This will take up to a minute, please wait...)</span>')

try:
  from exotic.api.colab import *
  from exotic.api.plotting import plot_image
except ImportError:
  !pip install exotic --upgrade
  #!pip install git+https://github.com/alienlifeform/exotic-prototype.git --upgrade
  # This suppresses the "RESTART RUNTIME" button
  hideWarning()
  appendStepToContainer('.step_container_1a','<div class="attention"><b>Attention:</b> Due to installing new packages, please re-run this step by clicking the run arrow above again.</div>')
  hideWarning()
  import os
  os._exit(00)

# Import other utilities
appendStepToContainer('.step_container_1a','(3/5) NASAExoplanetArchive, Astropy, Utils')
from exotic.exotic import NASAExoplanetArchive, get_wcs, find_target
#from astropy.time import Time
#from barycorrpy import utc_tdb
#import numpy as np
#from io import BytesIO
#from astropy.io import fits
#from scipy.ndimage import label
#from bokeh.plotting import figure, output_file, show
#from bokeh.palettes import Viridis256
#from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, LogTicker
#from bokeh.models import BoxZoomTool,WheelZoomTool,ResetTool,HoverTool,PanTool,FreehandDrawTool
#from pprint import pprint
from IPython.display import Image
from ipywidgets import widgets, HBox
#from skimage.transform import rescale, resize, downscale_local_mean
#import copy
import os
import re
import json
#import subprocess
import glob

# Import matlab/stats (perhaps not necessary anymore)
appendStepToContainer('.step_container_1a','(4/5) Matlab, SciPy')
#import matplotlib.pyplot as plt
#from scipy.stats import gaussian_kde
showProgress(.3)

# Import Google Utils
appendStepToContainer('.step_container_1a','(5/5) Google Utils')
from google.colab import drive, files
showProgress(.3)

display(HTML('<p class="bookend">DONE: Importing necessary software libraries</p>'))

# Prepare user for loading of images
display(HTML('<p class="bookend">START: Mounting Google Drive</p>'))
display(HTML('<ul class="step_container_1b"></ul>'))
appendStepToContainer('.step_container_1b','<div class="attention"><b>Attention:</b> Be sure to "Permit this notebook to access your Google Drive files" if prompted.</div>')
showProgress(1)

# Mount the user's drive so we can access images
drive.mount('/content/drive', force_remount=True)

appendStepToContainer('.step_container_1b','Drive successfully mounted')

display(HTML('<p class="bookend">DONE: Mounting Google Drive.  <b>You may move on to the next step.</b></p>'))

---

## Step 2: Load Telescope Images

<font face="Helvetica, Arial, Sans-Serif">


🕔&nbsp;&nbsp;<i>Estimated time: 1 minute</i>
<br />

<br />


This step will:


<ol type="a">
<li class="step">Prompt you to type in the path to your .FITS files on Google Drive </li>
<li class="step">Prompt you to type in your AAVSO Observer code, if you have one.</li>
</ol>


For more information:


<ul><li class="step"><a href="https://exoplanets.nasa.gov/exoplanet-watch/exotic/user-guide/">How do I upload .FITS images to Google Drive and determine the path?</a></li>
<li class="step"><a href="https://www.aavso.org/new-observers#:~:text=If%20you%20do%20not%20yet,for%20%22Request%20an%20obscode%22.">How do I request an AAVSO Observer code?</a></li>
</ul>

</font>

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8><b>Load telescope images</b></font>

##############################################################
%%capture step_capture --no-display
# Comment the above statement out to see debugging information
##############################################################

##############################################################
#
# NOTE TO EXOTIC USER:
#
#   • To hide this code, double-click the title above ("Load telescope images"),
#     or click the arrow to the left of the title.
#
#   • Editing this code will only affect your local instance.
#     Reload to revert your changes.
#
##############################################################

setupDisplay()

# Prepare user for loading of images
display(HTML('<p class="bookend">START: Loading telescope images</p>'))
display(HTML('<ul class="step_container_2a"></ul>'))
appendStepToContainer('.step_container_2a','Ensuring images are loaded...</li>')
showProgress(1)

def clean_input_filepath(p):
  p = re.sub('^/content', '', p)
  p = re.sub('^/drive/MyDrive', '', p)
  p = re.sub('^/', '', p)
  return(p)

######################################################


expandableSectionCustom('<u>+ EXOTIC Inline Help:</u> How to Upload your .FITS images into Google Drive in way that EXOTIC can use them','<u>- Close EXOTIC Inline Help</u>','''
  <p><b>How to upload your .FITS images into Google Drive in way that EXOTIC can use them:</b></p>
  <blockquote>e.g. EXOTIC/HatP32Dec202017/</blockquote>

  <ol style="line-height:135%">
  <li>In another window, <a href="https://drive.google.com/drive/my-drive" target="newGoogleDrive">go to Google Drive</a>.</li>
  <li><u>In Google Drive</u>, <i>if you don't already have an EXOTIC folder</i>, right click on "My Drive" (in the left nav) and click New Folder. Name the folder "EXOTIC".</li>
  <li>Click the arrow next to "My Drive" to see the subfolders and click "EXOTIC".</li>
  <li><u>On your computer</u>, put your .FITS files into a single folder uniquely named for your observation (e.g. "HatP32Dec202017").</li>
  <li>From your filesystem, drag this folder into Google Drive where it says "Drop files here".</li>
  </ol>

  <p>You will use this path (e.g. "EXOTIC/HatP32Dec202017") when loading your images into EXOTIC.</p>
''')


expandableSectionCustom('<u>+ EXOTIC Inline Help:</u> How to find your .FITS images','<u>- Close EXOTIC Inline Help</u>','''
  <p><b>How to find your .FITS images:</b></p>

  <ol style="line-height:135%">
  <li>In the left nav, click on the folder icon.</li>
  <li>Navigate to your images, likely in /drive/MyDrive/ or /content/drive/MyDrive/.</li>
  <li class="error">If you click the "up one directory" icon, you will see a long list of folders. Click "content"</li>
  <li>Right-click on the folder with your .FITS images and click "Copy Path".</li>
  </ol>

  <p>You will use this path (e.g. "EXOTIC/HatP32Dec202017" or "/content/drive/MyDrive/EXOTIC/HatP32Dec202017") when loading your images into EXOTIC.</p>
''')

# Ask for inputs until we find .fits files
fits_files_found = False
while not fits_files_found:
  # Ask for inputs until we find ANY files
  uploaded_files_found = False
  #appendStepToContainer('.step_container_2a','A valid Google Drive filepath should not include /drive/MyDrive/')
  while not uploaded_files_found:
    input_filepath = input('Enter path to .FITS images in Google Drive (i.e. "EXOTIC/HatP32Dec202017") and press return:  ')
    #display(HTML(f'<p class="output">input_filepath={input_filepath}</p>'))
    cleaned_filepath = clean_input_filepath(input_filepath)
    #display(HTML(f'<p class="output">cleaned_filepath={cleaned_filepath}</p>'))
    if cleaned_filepath:
      verified_filepath = check_dir(os.path.join("/content/drive/My Drive/", cleaned_filepath))
      #display(HTML(f'<p class="output">verified_filepath={verified_filepath}</p>'))

      if verified_filepath:
        output_dir = verified_filepath + "_output/"
        #display(HTML(f'<p class="output">output_dir={output_dir}</p>'))

        sorted_files = sorted(os.listdir(verified_filepath));
        #display(HTML(f'<p class="output">sortedFiles={sorted_files}</p>'))

        if sorted_files:
          uploaded_files_found = True # exit inner loop and continue
        else:
          display(HTML(f'<p class="error">Failed to find files at {verified_filepath}. You can click the folder icon in the left nav to browse your Google Drive directories.</p>'))
      else:
        display(HTML(f'<p class="error">Failed to find a folder at /content/drive/My Drive/{cleaned_filepath}. You can click the folder icon in the left nav to browse your Google Drive directories.</p>'))
    else:
      display(HTML(f'<p class="error">Filepath doesn\'t seem right: /content/drive/My Drive/{input_filepath}. You can click the folder icon in the left nav to browse your Google Drive directories.</p>'))


  # Directory full of files found, look for .fits and inits.json
  uploaded_files = [f for f in sorted_files if os.path.isfile(os.path.join(verified_filepath, f))]
  fits_count, inits_count, first_image = 0, 0, ""


  # Identify .FITS and inits.json files in user-submitted folder
  inits = []    # array of paths to any inits files found in the directory
  fits_list = []
  for f in uploaded_files:
    # Look for .fits images and keep count
    if f.lower().endswith(('.fits', '.fits.gz', '.fit')):
      fits_list.append(f)
      if first_image == "":
        first_image = os.path.join(verified_filepath, f)
      fits_count += 1
    # Look for inits.json file(s)
    if re.search(r"\.json$", f, re.IGNORECASE):
      inits.append(os.path.join(verified_filepath, f))

  inits_count = len(inits)
  display(HTML(f'<p class="output"><br />Found {fits_count} image files and {inits_count} initialization files in the directory.</p>'))

  # Determine if folder has enough .FITS folders to move forward
  if fits_count >= 1:
    fits_files_found = True # exit outer loop and continue

    # Make the output directory if it does not exist already.
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
      display(HTML(f'<p class="output">Creating output_dir at {output_dir}</p>'))
    output_dir_for_shell = output_dir.replace(" ", "\ ")
  else:
    display(HTML(f'<p class="error">Failed to find a significant number of .FITS files at {verified_filepath}</p>'))

# Read configuration from inits.json, if available
if inits_count == 1:                 # one inits file exists
  # Deal with inits.json file
  inits_file_path = os.path.join(verified_filepath, inits[0])
  inits_file_exists = True
  #display(HTML(f'<p class="output">Got an inits.json file here: {inits_file_path}</p>'))
  with open(inits_file_path) as i_file:
    display(HTML(f'<p class="output">Loading coordinates and input/output directories from inits file</p>'))
    inits_data = i_file.read()
    d = json.loads(inits_data)
    targ_coords = d["user_info"]["Target Star X & Y Pixel"]
    comp_coords = d["user_info"]["Comparison Star(s) X & Y Pixel"]
    input_dir = d["user_info"]["Directory with FITS files"]
    if input_dir != verified_filepath:
      display(HTML(f'<p class="error">The directory with fits files should be {verified_filepath} but your inits file says {input_dir}.</p>'))
      display(HTML('<p class="output">This may or may not cause problems.  Just letting you know.<p>'))
    display(HTML(f'<p class="output">Coordinates from your inits file:\ntarget: {targ_coords}\ncomps: {comp_coords}<p>'))
    output_dir = d["user_info"]["Directory to Save Plots"]
else:
  display(HTML(f'<p class="output">No valid inits.json file was found, we\'ll create it in the next step.<p>'))
  inits_file_exists = False

showProgress(1)
display(HTML('<p class="bookend">DONE: Loading telescope images</p>'))


######################################################

# Load planetary params if inits.json file does not yet exist
if not inits_file_exists:

  display(HTML('<p class="bookend">START: Download planetary parameters</p>'))

  planetary_params = ""
  while not planetary_params:
    target_is_valid = False
    while not target_is_valid:
      target=input('Please enter the name of your exoplanet target (i.e. "HAT-P-32 b") and press return: ')
      if target != "":
        target_is_valid = True
      else:
        display(HTML(f'<p class="error">Exoplanet target may not be blank.</p>'))
        starchart_image_url_is_valid = False

    display(HTML('<br /><ul class="step_container_2b"></ul>'))
    appendStepToContainer('.step_container_2b','Searching NASA Exoplanet Archive for "' + target + '"...')

    targ = NASAExoplanetArchive(planet=target)
    #appendStepToContainer('.step_container_2','Loading planet info')
    target = targ.planet_info()[0]

    if not targ.resolve_name():
      appendStepToContainer('.step_container_2b','''
      Sorry, we can\'t find your target in the Exoplanet Archive.  Unfortunately, this
      isn't going to work until we can find it. Please try
      different formats for your target name, until the target is located.
      Looking it up in the NASA Exoplanet Archive at https://exoplanetarchive.ipac.caltech.edu/
      might help you know where to put the spaces and hyphens and such.
      ''')
      appendStepToContainer('.step_container_2b','''
      If your target is a candidate, you may need to create your own inits.json file and
      add it to the folder with your FITS images.
      ''')
    else:
      appendStepToContainer('.step_container_2b','Found target "' + target + '" in the NASA Exoplanet Archive')
      p_param_string = targ.planet_info(fancy=True)
      planetary_params = '"planetary_parameters": ' + p_param_string
      p_param_dict = json.loads(p_param_string)
      planetary_params = fix_planetary_params(p_param_dict)
      appendStepToContainer('.step_container_2b','Loading NASA Exoplanet Archive planetary parameters for ' + target)
      display(HTML(f'<pre class="output">{planetary_params}</pre>'))


  expandableSectionCustom('<u>+ EXOTIC Inline Help:</u> How to get an AAVSO Observer code','<u>- Close EXOTIC Inline Help</u>','''
    <p><b>How to get an AAVSO Observer code:</b></p>
    Follow the instructions at the <a href="https://www.aavso.org/new-observers#:~:text=If%20you%20do%20not%20yet,for%20%22Request%20an%20obscode%22." target="_blank">AAVSO "New Observers" page</a>.
  ''')

  # Prompt for AAVSO code
  aavso_obs_code = input("Enter your AAVSO Observer code or press return to skip: ")
  if aavso_obs_code:
    sec_obs_code = input("Enter a secondary AAVSO Observer code or press return to skip: ")
  else:
    sec_obs_code = ""

  display(HTML('<p class="bookend">DONE: Download planetary parameters. <b>You may move on to the next step.</b></p>'))

else:

  display(HTML('<p class="bookend">DONE: Inits.json file exists. <b>You may move on to step 4.</b></p>'))

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8><b>Clean images (optional)</b></font>

##############################################################
%%capture step_capture --no-display
# Comment the above statement out to see debugging information
##############################################################

setupDisplay()
display(HTML('<ul class="step_container_2c"></ul>'))


# IMAGE CLEANING STEP
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from astropy.visualization import ImageNormalize, ZScaleInterval
import matplotlib.pyplot as plt
import shutil


b = False
while b == False:
    answer = input("Would you like to manually check image quality? [y/n] \n")
    #display(HTML(f'<p class="output">answer={answer}</p>'))

    if answer.lower() == 'y':
        do_image_proc = True
        b = True
    elif answer.lower() == 'n':
        do_image_proc = False
        b = True
    else:
        print('Invalid response.')


if do_image_proc:

    output_area = widgets.Output()
    display(output_area)

    def display_fits_with_remove(directory_path, fits_files):
        """
        Function that displays fits images one at a time, with interactive buttons to keep or delete,
        and an undo functionality.

        Parameters:
        directory_path (str): path to the directory containing the fits files.
        fits_files (list): list of fits files to be reviewed.

        Returns:
        None
        """
        # Make bad images directory
        bad_dir = os.path.join(directory_path, 'Bad Images')
        if not os.path.exists(bad_dir):
            os.mkdir(bad_dir)

        # Define the keep, delete, and undo buttons
        keep_button = widgets.Button(description='Keep', button_style='success', layout=widgets.Layout(width='20%'))
        delete_button = widgets.Button(description='Delete', button_style='danger', layout=widgets.Layout(width='20%'))
        undo_button = widgets.Button(description='Undo', button_style='info', layout=widgets.Layout(width='20%'))
        done_button = widgets.Button(description='Stop', button_style='warning', layout=widgets.Layout(width='20%'))

        # Define the image display area
        image_area = widgets.Output()

        # Define the main container
        main_container = VBox([image_area, HBox([keep_button, delete_button, undo_button, done_button],
                                              layout=widgets.Layout(justify_content='flex-start',
                                                            flex_flow='row wrap',
                                                            align_items='stretch',
                                                            width='60%'))])

        # Define the button click actions
        def on_keep_button_clicked(b):
            nonlocal i
            with output_area:
                output_area.clear_output(wait=True)
                display(HTML(f"<p class='output'>{fits_files[i]} is kept.</p>"))
            image_area.clear_output(wait=True)
            i += 1
            next_image()

        def on_delete_button_clicked(b):
            nonlocal i, prev_i, prev_image_path
            image_path = os.path.join(directory_path, fits_files[i])
            bad_image_path = os.path.join(directory_path, 'Bad Images', fits_files[i])
            shutil.move(image_path, bad_image_path)
            with output_area:
                output_area.clear_output(wait=True)
                display(HTML(f"<p class='output'>{fits_files[i]} is deleted.</p>"))
            image_area.clear_output(wait=True)
            prev_i = i
            i += 1
            next_image()

        def on_undo_button_clicked(b):
            nonlocal i, prev_image_path, prev_i
            if i > 0:
                prev_bad_image_path = os.path.join(directory_path, 'Bad Images', fits_files[prev_i])
                if os.path.exists(prev_bad_image_path):
                    original_image_path = os.path.join(directory_path, fits_files[prev_i])
                    if os.path.exists(original_image_path):
                      suffix = 1
                      while os.path.exists(original_image_path[:-5] + f"_{suffix}.fits"):
                            suffix += 1
                      original_image_path = original_image_path[:-5] + f"_{suffix}.fits"
                    shutil.move(prev_bad_image_path, original_image_path)
                    with output_area:
                        output_area.clear_output(wait=True)
                        display(HTML(f"<p class='output'>{fits_files[prev_i]} is recovered.</p>"))
                    image_area.clear_output(wait=True)
                    i = prev_i
                    next_image()
                else:
                    with output_area:
                        output_area.clear_output(wait=True)
                        display(HTML("<p class='output'>Previous image was not deleted.</p>"))
            else:
                with output_area:
                      output_area.clear_output(wait=True)
                      display(HTML("<p class='output'>Cannot undo the first image.</p>"))

        def on_done_clicked(b):
          with output_area:
                    output_area.clear_output(wait=True)
                    display(HTML("<p class='output'>Done reviewing fits files. Continue to next step.</p>"))
          main_container.close()

        # Image cycling function
        def next_image():
            nonlocal i, prev_image_path, prev_i
            if i < len(fits_files):
                image_path = os.path.join(directory_path, fits_files[i])
                if os.path.exists(image_path):
                    with fits.open(image_path) as hdul:
                        image_data = hdul[0].data
                        norm = ImageNormalize(image_data, interval=ZScaleInterval(), vmin=np.nanpercentile(image_data, 5),
                                              vmax=np.nanpercentile(image_data, 99))
                        fig, ax = plt.subplots()
                        ax.imshow(image_data, cmap='viridis', norm=norm)
                        ax.axis('off')
                        ax.set_title("Image {}/{}".format(i + 1, len(fits_files)))
                        # Convert the plot to an image
                        with BytesIO() as output:
                            fig.savefig(output, dpi=100, bbox_inches='tight', pad_inches=0, format='png', transparent=True)
                            image_bytes = output.getvalue()
                        # Close the figure to release the memory
                        plt.close(fig)
                        # Display the image in the widget
                        with image_area:
                            image_area.clear_output(wait=True)
                            display(widgets.Image(value=image_bytes))
                    prev_image_path = image_path
                    return
            else:
                with output_area:
                    output_area.clear_output(wait=True)
                    display(HTML("<p class='output'>Done reviewing fits files. Continue to next step.</p>"))
                main_container.close()

        # Initialize the index variables
        i = 0
        prev_i = 0
        prev_image_path = None

        # Display the first image
        next_image()

        # Assign the button click actions
        keep_button.on_click(on_keep_button_clicked)
        delete_button.on_click(on_delete_button_clicked)
        undo_button.on_click(on_undo_button_clicked)
        done_button.on_click(on_done_clicked)

        display(main_container)


    display_fits_with_remove(verified_filepath, fits_list)

else:
    display(HTML("<p class='output'>Continue to next step.</p>"))


---

## Step 3: Identify target star and comparison stars

<font face="Helvetica, Arial, Sans-Serif">


🕔&nbsp;&nbsp;Estimated time: 3 minutes
<br />

<br />


This step will:


<ol type="a">
<li class="step">Prompt you to type in your starchart image URL</li>
<li class="step">Ask you to identify coordinates for your target star and comparison stars</li>
</ol>



For more information:

<ul>
<li class="step"><a href="https://exoplanets.nasa.gov/exoplanet-watch/exotic/user-guide/#2-2">How do I get a starchart image URL?</a></li>
<li class="step"><a href="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.html" target="_blank">How do I identify coordinates for my target and comparison stars?</a>
</li>
</ul>

</font>

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8><b>Identify target and comparison stars in a telescope image</b></font>

##############################################################
#
# NOTE TO EXOTIC USER:
#
#   • To hide this code, double-click the title above the code,
#     or click the arrow to the left of the title.
#
#   • Editing this code will only affect your local instance.
#     Reload to revert your changes.
#
##############################################################

#@markdown <font face="Helvetica, Arial, Sans-Serif" size=2>
#@markdown Select your telescope and target host star (e.g. "HAT-P-32"), then press <img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABUAAAAVCAYAAACpF6WWAAABV2lDQ1BJQ0MgUHJvZmlsZQAAKJFjYGBiSSwoyGFhYGDIzSspCnJ3UoiIjFJgf8bAycABhEIMbInJxQWOAQE+QCUMMBoVfLvGwAiiL+uCzJr3s/Z77Blu8WhFRuc9VUV3MNWjAK6U1OJkIP0HiLWSC4pKGBgYNYDsgPKSAhC7AsgWKQI6CsjuAbHTIewFIHYShL0FrCYkyBnIPgFkCyRnJKYA2TeAbJ0kJPF0JHZuTmky1A0g1/Ok5oUGA2k+IJZh8GAIYFBgMALCSoYMYNhgV2sCVuvMkM9QAFRXxJDJkA5UXQLU6QgUKWDIYUgFsj0Z8hiSGfQYdMAmGoBMBYUxetghxPIXMTBYfGVgYJ6AEEuaycCwvZWBQeIWQkwFGAb8LQwM284XJBYlwkOU8RtLcZqxEYTN48TAwHrv///PagwM7JMZGP5O+P//96L///8uBpoPjJsDeQDaFGLQdABjBAAAAFZlWElmTU0AKgAAAAgAAYdpAAQAAAABAAAAGgAAAAAAA5KGAAcAAAASAAAARKACAAQAAAABAAAAFaADAAQAAAABAAAAFQAAAABBU0NJSQAAAFNjcmVlbnNob3TrIl/MAAAB1GlUWHRYTUw6Y29tLmFkb2JlLnhtcAAAAAAAPHg6eG1wbWV0YSB4bWxuczp4PSJhZG9iZTpuczptZXRhLyIgeDp4bXB0az0iWE1QIENvcmUgNi4wLjAiPgogICA8cmRmOlJERiB4bWxuczpyZGY9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkvMDIvMjItcmRmLXN5bnRheC1ucyMiPgogICAgICA8cmRmOkRlc2NyaXB0aW9uIHJkZjphYm91dD0iIgogICAgICAgICAgICB4bWxuczpleGlmPSJodHRwOi8vbnMuYWRvYmUuY29tL2V4aWYvMS4wLyI+CiAgICAgICAgIDxleGlmOlBpeGVsWURpbWVuc2lvbj4yMTwvZXhpZjpQaXhlbFlEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4yMTwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlVzZXJDb21tZW50PlNjcmVlbnNob3Q8L2V4aWY6VXNlckNvbW1lbnQ+CiAgICAgIDwvcmRmOkRlc2NyaXB0aW9uPgogICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgqn5DzTAAABCUlEQVQ4Ea2UQQ6EIAxFdTIXYu1tdOtx2OptXHukmflkPmkrQklsQoAOffyWjuPnZ8PD9nqYl3Bu6HmeA4bH3rVD27YlkIWFEAaMZVmK4WOppoDt+14MsM4YY7pA+i/QHiBB8zwr1aqmSNOrkEDMiIEYmlI6TRP9akb9bF3Vgf/mOI60ykrlTTYAD4IUW0ZGhraUAFx6FHkRGW4oglGGGlhBuZG31tYtcFIKBT2G2t0JASv/ozwvDFit5Sgu15SOO8UtIOLIcPXp3UXWf+lTHPD0ogVxj8ej5fTh8DY5gzlDDFOHT6XPQ3jZdV25rc72Y4LDRSgpbB3bPlCF0fU9JVTOBMs05e9y/QUaNJJ8hZpK/QAAAABJRU5ErkJggg=="> to the left (or shift-return).
#@markdown </font>
Telescope = 'MicroObservatory' #@param ["Select a Telescope", "MicroObservatory", "Exoplanet Watch .4 Meter"]
Star = 'Qatar-1' #@param {type:"string"}
Target = Star

j = 0
while not os.path.exists(first_image):
  first_image = verified_filepath + '/' + fits_list[j]
  j += 1

setupDisplay()

#########################################################

import urllib
from urllib.request import urlopen
from urllib.error import HTTPError

def get_star_chart_urls(telescope, star_target):
  if telescope == 'MicroObservatory':
    t_fov=56.44
    t_maglimit=15
    t_resolution=150
  elif telescope == 'Exoplanet Watch .4 Meter':
    t_fov=38.42
    t_maglimit=15
    t_resolution=150
  else:
    # Should not get here
    t_fov=56.44
    t_maglimit=15
    t_resolution=150
  json_url = f"https://app.aavso.org/vsp/api/chart/?star={star_target}&scale=D&orientation=CCD&type=chart&fov={t_fov}&maglimit={t_maglimit}&resolution={t_resolution}&north=down&east=left&format=json"
  starchart_url = f"https://app.aavso.org/vsp/?star={star_target}&scale=D&orientation=CCD&type=chart&fov={t_fov}&maglimit={t_maglimit}&resolution={t_resolution}&north=down&east=left"
  return [json_url, starchart_url]

def get_star_chart_image_url(json_url):
  display(HTML(f'<p class="output">Searching for startchart at {json_url}</p>'))
  with urllib.request.urlopen(json_url) as url:
    starchart_data = json.load(url)
    image_uri = starchart_data["image_uri"].split('?')[0]
    display(HTML(f'<p class="output">Pulling starchart JSON to find image url... found: {image_uri}</p>'))
    return(image_uri)

######################################################


if not inits_file_exists:
  if Telescope == 'Select a Telescope' or not Target:
    display(HTML('<span class="error">You must enter a Telescope and Target Host Star (e.g. "HAT-P-32") to run this step.</span>'))
  else:
    display(HTML('<p class="bookend">START: Generate AAVSO StarChart</p>'))

    #display(HTML('<br /><p><b>Option A:</b> Enter a telescope and target star to generate an AAVSO starchart image URL.'))


    starchart_image_url = ''
    starchart_image_url_is_valid = False
    prompt_for_url = True

    display(HTML(f'<p class="output">Telescope was selected: {Telescope}</p>'))
    display(HTML(f'<p class="output">Target Host Star was selected: {Target}</p>'))

    starchart_urls = get_star_chart_urls(Telescope,Target)
    try:
      # Generate the starchart image url
      starchart_image_url = get_star_chart_image_url(starchart_urls[0])
      #display(HTML(f'<p class="output">starchart_image_url: {starchart_image_url}</p>'))
      starchart_image_url_is_valid = True
      prompt_for_url = False
    except HTTPError:
      display(HTML('<p class="error">Could not find a starchart matching that target host star.</p>'))
      display(HTML(f'<p class="error">Try a different target host star (make sure it\'s a star, not a planet) and click "run" once to stop, and again to re-run this step.</p>'))
      display(HTML(f'<p class="output">Or... <a href="{starchart_urls[1]}" target="_blank">use the advanced search on AAVSO</a> to find the URL of the image associated with your starchart.</p>'))
      prompt_for_url = True


    while prompt_for_url:
      #while not starchart_image_url_is_valid:
      starchart_image_url = input('Enter a valid starchart image URL and press return: ')
      if starchart_image_url.startswith('https://') and starchart_image_url.endswith('png'):
        display(HTML(f'<p class="output">Starchart Image URL is valid: {starchart_image_url}</p>'))
        starchart_image_url_is_valid = True
        prompt_for_url = False
      else:
        display(HTML(f'<p class="error">Starchart Image URL must begin with https:// and end with .png: {starchart_image_url}</p>'))
        display(HTML(f'<p class="output">If you found a starchart, click on the image and then grab the URL from your browser\'s URL bar.</p>'))
        starchart_image_url_is_valid = False

    ######################################################

    #display_images_for_comparision(first_image, starchart_image_url)

    display(HTML('<p class="bookend">DONE: Generate AAVSO StarChart</p>'))

    showProgress(2)


    if fits_files_found and starchart_image_url_is_valid:

      #########################################################

      display(HTML('<p class="bookend">START: Compare telescope image and starchart</p>'))
      display(HTML('<ul class="step_container_3b">'))

      # set up bokeh
      bokeh.io.output_notebook()
      sample_data = False

      # set up image path
      #p = "sample-data/HatP32Dec202017"
      #p = os.path.join("/content/EXOTIC/exotic-in-action/", p)
      #p = check_dir(os.path.join("/content/drive/My Drive/EXOTIC/MyOwnImages/", p))
      #output_dir = os.path.join(p, "EXOTIC_output/")

      #########################################################

      # show images
      if first_image:
        obs = ""
        # instructions for finding the target star
        display(HTML(f'''
          <h3>Data Entry 1 of 2: Enter coordinates for the target star</h3>
          <ol class="step">
            <li class="step">In the right image, find the <i>crosshairs</i> in the center - that represents your target star.</li>
            <li class="step">On the left image, <i>find this target star and roll over it with your mouse</i>, note the X and Y coordinates.</li>
            <li class="step">Put the X and Y coordinates in the box below in the format <code>[x,y]</code> and press return.</li>
          </ol>
          <p>Tip: Use the zoom feature. Click the magnifying glass and click-and-drag to draw a rectangle that matches the starchart.</p>

        '''))
        showProgress(2)
        display(HTML(f'''
          <div class="plots">

          <img class="aavso_image" src="{starchart_image_url}">
        '''))
        display_image(first_image)
        display(HTML('</div><br clear="all"/>'))

        showProgress(3)

        # request coordinates and verify the entries are valid
        success = False
        while not success:
          targ_coords = input("Enter coordinates for target star - in the format [424,286] - and press return:  ")

          # check syntax and coords
          targ_coords = targ_coords.strip()
          tc_syntax = re.search(r"\[\d+, ?\d+\]$", targ_coords)
          if tc_syntax:
              success = True
          else:
            display(HTML(f'<p class="error">Try again, your syntax is not quite right: {targ_coords} needs to look like [424,286]</p>'))


        showProgress(2)

        # instructions for finding the comparison stars
        display(HTML('''
          <p class="output">Target star coordinates saved to inits.json</p>
          <h3>Data Entry 2 of 2: Enter coordinates for at least two comparison stars.</h3>
          <ol class="step">
            <li class="step">In the right image, find the stars <i>with numbers</i> that represent suggested comparison stars.</li>
            <li class="step">On the left image, <i>find and roll over each comparison star with your mouse</i> and note the coordinates.</li>
            <li class="step">Put the X and Y coordinates in the box below in the format <code>[[x1,y1],[x2,y2]]</code> and press return.</li>
          </ol>
          <h4> Note: if there are no comparison stars identified in your field, please select ~3-5 stars that are relatively close to your target star and that are as bright, if not brighter, than your target star.</h4>
        '''))

        # request coordinates and verify the entries are valid
        success = False
        while not success:
          comp_coords = input("Enter coordinates for the comparison stars - in the format [[326,365],[416,343]] - and press return:  ")

          # check syntax
          comp_coords = comp_coords.strip()
          cc_syntax = re.search(r"\[(\[\d+, ?\d+\],? ?)+\]$", comp_coords)
          if cc_syntax:
            #display(HTML(f'<p class="output">Syntax OK:  [[x1,y1],[x2,y2]] e.g. {comp_coords}</p>'))
            success = True
          else:
            display(HTML(f'<p class="error">Try again, your syntax is not quite right: {comp_coords} needs to look more like [[326,365],[416,343]]</p>'))

        display(HTML('<p class="output">Coordinates submitted. Creating initialization file "inits.json"</p>'))

        inits_file_path = make_inits_file(planetary_params, verified_filepath, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sec_obs_code, sample_data)
        showProgress(1)

        if inits_file_path:
          display(HTML('<p class="bookend">DONE: Compare telescope image and starchart. <b>You may move on to the next step.</b></p>'))
        else:
          display(HTML('<p class="error">Warning: No inits.json path created. This is unexpected.</p>'))

      else:
        display(HTML('<p class="bookend">DONE: First .FITS image not found. <b>Please go back to step 2 and load your .FITS images.</b></p>'))

    else:
      display(HTML('<p class="bookend">DONE: .FITS files not loaded. <b>Please go back to step 2 and load your .FITS images.</b></p>'))

else:
  display(HTML(f'''
    <p class="bookend">No need to run this step, since your inits.json file already exists at {inits_file_path}.
    <br /><b>Move on to step 4,</b> or remove the file and re-run steps 2 and 3 to generate a new one.</p>
    '''))

---

## Step 4: Run EXOTIC to generate a lightcurve

<font face="Helvetica, Arial, Sans-Serif">


🕔&nbsp;&nbsp;Estimated time: 3-15 minutes
<br />

<br />


This step will:


<ol type="a">
<li class="step">Run EXOTIC to generate a lightcurve that you can download and submit to AAVSO</li>
</ol>

</font>

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8><b>Run EXOTIC to analyze telescope images</b></font>

##############################################################
#
# NOTE TO EXOTIC USER:
#
#   • To hide this code, double-click the title above the code,
#     or click the arrow to the left of the title.
#
#   • Editing this code will only affect your local instance.
#     Reload to revert your changes.
#
##############################################################

setupDisplay()

# If the user presses enter to run the sample data, download sample data if needed and
# put it into a sample-data directory at the top level of the user's Gdrive.  Count
# the .fits files (images) and .json files (inits files) in the directory entered
# by the user (or in the sample-data directory if the user pressed enter).  If
# there are at least 20 .fits files, assume this is a directory of images and display
# the first one in the series.  If there is exactly one inits file in the directory,
# show the specified target and comp coords so that the user can check these against
# the displayed image.  Otherwise, prompt for target / comp coords and make an inits
# file based  on those (save this new inits file in the folder with the output files
# so that the student can consult it later).  Finally, run EXOTIC with the newly-made
# or pre-existing inits file, plus any other inits files in the directory.

#########################################################

if 'inits_file_path' in globals():

  # p is the name of the folder entered by the user.  Decide what to do based on what
  # is found in the folder.
  display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
  #display(HTML("<p class='warning'>NOTE: At this point in EXOTIC, you would have the opportunity choose where to temporarily save the sample data. For this exercise, we're downloading to /content/EXOTIC/exotic-quick-start/sample-data/HatP32Dec202017"))
  display(HTML('<ul class="step5">'))

  #bokeh.io.output_notebook()
  sample_data = False

  print("Path to the inits file(s) that will be used: " + inits_file_path)

  commands = []
  with open(inits_file_path) as i_file:
    inits_data = i_file.read()
    d = json.loads(inits_data)
    date_obs = d["user_info"]["Observation date"]
    planet = d["planetary_parameters"]["Planet Name"]
    output_dir = d["user_info"]["Directory to Save Plots"]
    if not os.path.isdir(output_dir):
      os.makedirs(output_dir)
    inits_file_for_shell = inits_file_path.replace(" ", "\\ ")
    run_exotic = str(f"exotic -red {inits_file_for_shell} -ov")
    debug_exotic_run = str(f"!exotic -red \"{inits_file_path}\" -ov")

    commands.append({"inits_file_for_shell": inits_file_for_shell, "output_dir": output_dir,
                      "planet": planet, "date_obs": date_obs,
                      "run_exotic": run_exotic, "debug_exotic_run": debug_exotic_run
                      })
    print(f"{debug_exotic_run}")
    !eval "$run_exotic"

    file_for_submission = os.path.join(output_dir,"AAVSO_"+planet+"_"+date_obs+".txt")
    lightcurve = os.path.join(output_dir,"FinalLightCurve_"+planet+"_"+date_obs+".png")
    fov = os.path.join(output_dir,"temp/FOV_"+planet+"_"+date_obs+"_LinearStretch.png")
    triangle = os.path.join(output_dir,"temp/Triangle_"+planet+"_"+date_obs+".png")

    print(f"aavso output: {file_for_submission}\nlightcurve: {lightcurve}\nfov: {fov}\ntriangle: {triangle}")

    if not (os.path.isfile(lightcurve) and os.path.isfile(fov) and os.path.isfile(triangle)):
      print(f"Something went wrong with {planet} {date_obs}.\nCopy the command below into a new cell and run to find the error:\n{debug_exotic_run}\n")

    imageA = widgets.Image(value=open(lightcurve, 'rb').read())
    imageB = widgets.Image(value=open(fov, 'rb').read())
    hbox = HBox([imageB, imageA])
    display(hbox)
    display(Image(filename=triangle))


  ###

  display(HTML('<p class="bookend">DONE: Analyzing telescope images. </p>'))

  showProgress(3)

  display(HTML(f'''

    <h2>Congratulations!</h2>
    <h3>You have successfully generated a lightcurve showing the possible transit of {planet}</h3>

    <li class="step">Click to download the data to your hard drive in a format suitable for submission to AAVSO, (or find it by clicking the folder icon in the left nav and navigating to {file_for_submission})</li>

  '''))


  # Allow download of lightcurve data
  def on_dl_button_clicked(b):
    # Display the message within the output widget.
    if os.path.isfile(file_for_submission):
      display(HTML('<p>Downloading lightcurve data...</p>'))
      showProgress(2)
      files.download(file_for_submission)
    else:
      display(HTML('<p>Couldn\'t find output file.</p>'))

  dl_button = widgets.Button(description="Download data")
  dl_button.on_click(on_dl_button_clicked)
  display(HTML('<br /><hr /><br />'))
  display(dl_button)

else:
  display(HTML('<p class="bookend">Couldn\'t find inits.json file. <b>Return to step 2.</b></p>'))

---

## End

<font face="Helvetica, Arial, Sans-Serif">


What would you like to do next?

<a href="https://exoplanets.nasa.gov/exoplanet-watch/exotic/user-guide/#5-1">Learn how to submit a lightcurve</a>



<a href="https://app.aavso.org/exosite/submit">Submit your lightcurve to Exoplanet Watch via AAVSO</a>

<a href="https://exoplanets.nasa.gov/exoplanet-watch/about-exoplanet-watch/contact-us/">Provide feedback for this tutorial</a>


</font>